In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/final-year-project/NeighRadius.csv
/kaggle/input/final-year-project/Radius.csv
/kaggle/input/final-year-project/Face.csv
/kaggle/input/final-year-project/NeighFace.csv


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import statsmodels.api as sm

In [3]:
df_face = pd.read_csv('/kaggle/input/final-year-project/Face.csv')
df_neigh_face= pd.read_csv('/kaggle/input/final-year-project/NeighFace.csv')
df_neigh_radius= pd.read_csv('/kaggle/input/final-year-project/NeighRadius.csv')
df_radius = pd.read_csv('/kaggle/input/final-year-project/Radius.csv')

In [4]:
merged_df = pd.concat([df_radius.set_index('Unnamed: 0'), df_face.set_index('Unnamed: 0'), df_neigh_radius.set_index('Unnamed: 0'), df_neigh_face.set_index('Unnamed: 0')], axis=1, keys=['Radius', 'Face', 'NeighRadius', 'NeighFace'])


In [5]:
time_point = 't1'
data_at_t1 = merged_df.xs(time_point, level=1, axis=1).reset_index()


In [6]:
data_at_t1_numeric = data_at_t1.drop(['Unnamed: 0'], axis=1)

In [7]:
# Compute the correlation matrix for numeric data only
correlation_matrix_numeric = data_at_t1_numeric.corr()

correlation_matrix_numeric

,Radius,Face,NeighRadius,NeighFace
Radius,1.000000,0.746277,0.585424,0.235521
Face,0.746277,1.000000,0.140430,-0.027427
NeighRadius,0.585424,0.140430,1.000000,0.600565
NeighFace,0.235521,-0.027427,0.600565,1.000000


In [8]:
# Assuming data_at_t1_numeric already contains only the columns for 'Radius', 'Face', 'NeighRadius', and 'NeighFace' at time point t1
X = data_at_t1_numeric[['Face', 'NeighRadius', 'NeighFace']]  # Independent variables
y = data_at_t1_numeric['Radius']  # Dependent variable (the one you want to predict)


In [9]:
X = sm.add_constant(X)


In [10]:
# Fitting the Ordinary Least Squares (OLS) model
model = sm.OLS(y, X).fit()

In [11]:
# Print out the statistics
model_summary = model.summary()
print(model_summary)

                            OLS Regression Results                            
Dep. Variable:                 Radius   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.795
Method:                 Least Squares   F-statistic:                     6465.
Date:                Tue, 12 Mar 2024   Prob (F-statistic):               0.00
Time:                        03:27:57   Log-Likelihood:                -5301.5
No. Observations:                5000   AIC:                         1.061e+04
Df Residuals:                    4996   BIC:                         1.064e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -6.2113      0.217    -28.665      

The output from the OLS regression analysis provides a detailed look into the relationship between the dependent variable `Radius` and the independent variables `Face`, `NeighRadius`, and `NeighFace`. The model's summary offers several key insights:

1. **Coefficients (coef)**: These values indicate the expected change in the dependent variable (grain radius) for a one-unit increase in the independent variable, holding all other variables constant.
   - **Face**: A coefficient of 0.3009 suggests that for each additional face count, the grain radius increases by approximately 0.3009 units.
   - **NeighRadius**: A coefficient of 1.2931 indicates that for each unit increase in the neighbor's average radius, the grain radius increases by approximately 1.2931 units.
   - **NeighFace**: A coefficient of -0.1219 means that for each additional neighbor face count, the grain radius decreases by approximately 0.1219 units.

2. **R-squared (0.795)**: This value indicates that approximately 79.5% of the variability in the grain radius can be explained by the model. It's a measure of how well the independent variables predict the dependent variable.

3. **Adjusted R-squared (0.795)**: Similar to the R-squared value, but it takes into account the number of predictors in the model. Since it's very close to the R-squared value, it suggests that the model is not penalized significantly for unnecessary complexity.

4. **F-statistic and Prob (F-statistic)**: The F-statistic value is quite large, and the probability of observing such a value if the independent variables had no relationship with the dependent variable is virtually zero. This strongly suggests that the model is significant.

5. **P>|t| (P-value)**: The p-values for all coefficients are significantly less than 0.05, indicating that each predictor is statistically significant, and we can reject the null hypothesis that a coefficient equals zero.

Based on these results, the regression equation that models the relationship between the variables is:

Radius = -6.2113 + 0.3009(Face) + 1.2931(NeighRadius) - 0.1219(NeighFace) ]

This equation can be used to predict the radius of a grain based on the face count, neighbor average radius, and neighbor face count. The negative coefficient for `NeighFace` suggests that having neighbors with a higher face count is associated with a smaller grain radius, perhaps due to constraints or competition in the polycrystalline structure. In contrast, the positive coefficients for `Face` and `NeighRadius` suggest that grains tend to be larger when they themselves have more faces and are surrounded by larger neighbors.